<a href="https://colab.research.google.com/github/ValdazoAmerico/generacion-tf-idf/blob/main/generacion_de_tf_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import nltk
import numpy as np
import random
import string

import bs4 as bs
import urllib.request
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
raw_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/Natural_language_processing')
raw_html = raw_html.read()

article_html = bs.BeautifulSoup(raw_html, 'lxml')

article_paragraphs = article_html.find_all('p')

article_text = ''
for para in article_paragraphs:
  article_text += para.text

corpus = nltk.sent_tokenize(article_text)

for i in range(len(corpus)):
  corpus[i] = corpus[i].lower()
  corpus[i] = re.sub(r'\W', ' ', corpus[i])
  corpus[i] = re.sub(r'\s+', ' ',corpus[i])

wordfreq = {}
for sentence in corpus:
  tokens = nltk.word_tokenize(sentence)
  for token in tokens:
    if token not in wordfreq.keys():
      wordfreq[token] = 1
    else:
      wordfreq[token] += 1


import heapq
most_freq = heapq.nlargest(200,wordfreq,key=wordfreq.get)

In [19]:
most_freq

['of',
 'the',
 'and',
 'to',
 'in',
 'language',
 'a',
 'natural',
 'is',
 'processing',
 'nlp',
 'as',
 'that',
 'cognitive',
 'machine',
 'learning',
 'for',
 'statistical',
 'with',
 'tasks',
 'on',
 'such',
 'linguistics',
 'rules',
 'e',
 'neural',
 'have',
 'models',
 'g',
 'more',
 'are',
 'based',
 'many',
 'has',
 'by',
 'systems',
 'algorithms',
 'been',
 'input',
 'data',
 'can',
 'speech',
 'an',
 'which',
 'however',
 'methods',
 'be',
 'research',
 'real',
 'they',
 'computer',
 'intelligence',
 'understanding',
 'documents',
 'from',
 'given',
 'or',
 'hand',
 'grammar',
 'part',
 'results',
 'increasingly',
 'used',
 'when',
 'since',
 'some',
 'science',
 'large',
 'task',
 'symbolic',
 '1980s',
 'most',
 'was',
 'this',
 'computational',
 'deep',
 'network',
 'set',
 'through',
 'world',
 'valued',
 'networks',
 'larger',
 'use',
 'approaches',
 'translation',
 'trends',
 'aspects',
 'artificial',
 'computers',
 'human',
 'how',
 'process',
 'analyze',
 'then',
 'wel

In [17]:
word_idf_values = {}
for token in most_freq:
  doc_containing_word = 0
  for document in corpus:
    if token in nltk.word_tokenize(document):
      doc_containing_word += 1
    word_idf_values[token] = np.log(len(corpus)/(1 + doc_containing_word))

In [25]:
word_tf_values = {}
for token in most_freq:
  sent_tf_vector = []
  for document in corpus:
    doc_freq = 0
    for word in nltk.word_tokenize(document):
      if token == word:
        doc_freq += 1
    word_tf = doc_freq/len(nltk.word_tokenize(document))
    sent_tf_vector.append(word_tf)
    
  word_tf_values[token] = sent_tf_vector

In [27]:
word_tf_values[token]

[0.0,
 0.0,
 0.045454545454545456,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [28]:
tfidf_values = []
for token in word_tf_values.keys():
  tfidf_sentences = []
  for tf_sentence in word_tf_values[token]:
    tf_idf_score = tf_sentence * word_idf_values[token]
    tfidf_sentences.append(tf_idf_score)
  tfidf_values.append(tfidf_sentences)

In [30]:
tf_idf_model = np.asarray(tfidf_values)

In [32]:
print(tf_idf_model)

[[0.01874153 0.05220854 0.         ... 0.03177911 0.02149763 0.        ]
 [0.00937076 0.06961139 0.04983542 ... 0.01588956 0.         0.        ]
 [0.0615983  0.         0.07279799 ... 0.         0.         0.        ]
 ...
 [0.         0.15231777 0.         ... 0.         0.         0.        ]
 [0.         0.         0.14539423 ... 0.         0.         0.        ]
 [0.         0.         0.14539423 ... 0.         0.         0.        ]]
